# Übungsaufgaben 3



## 1. Aufgabe `count_glyphs` (Encoding)
Schreiben Sie ein Programm (awk, Shell-Skript oder Python), das die
Anzahl der Buchstaben in einem Text zählt, wobei kombinierende Zeichen
nicht berücksichtigt werden. Das Programm soll von `stdin` lesen und
die Anzahl der Zeichen nach `stdout` schreiben (wie z.B. `wc`).

```bash
$ echo -n "Hallo aus München!" | uconv -f utf8 -t utf8 -x nfd | ./count_glyphs.py
18
$
```


#### Erläuterung

Der String `Hallo aus München!` enthält 18 Zeichen/Buchstaben, 17 davon ASCII-Zeichen. Der Umlaut `ü` kann entweder über ein Unicode-Zeichen repräsentiert werden (dann mit 2 Byte kodiert), oder als Kombination aus zweien (`u`+combining-character).

Wenn mit `nfd` diese zweite, dekomponierte Repräsentationsform erzwungen wird, zählt `wc` also insgesamt 19 Zeichen:

In [200]:
%%bash
echo -n "Hallo aus München!" | uconv -f utf8 -t utf8 -x nfd | wc -m

      19


Das  `counts_glyphs`-Programm soll diese Mitzählen kombinierender Zeichen verhindern.

### Lösung 1a: count_glyphs.bash mit uconv und nfc

- Verwendung von uconv zur Herstellung der komponierten Normalform (nfc) aus dem erzwungenen (nfd) dekomponierten Input, anschließend wc zur Zeichenzählung:

#### count_glyphs.bash:

In [ ]:
%%bash
#!/bin/bash
uconv -f utf8 -t utf8 -x nfc | wc -m

In [ ]:
# nfd: decomposition 
# nfc: composition

#### Usage:

In [189]:
%%bash
echo -n "Hallo aus München!" | uconv -f utf8 -t utf8 -x nfd | ./count_glyphs.py

18


### Lösung 1b: count_glyphs_mac.bash mit iconv und utf8-mac-Kodierung (nur Mac OS X, aber veraltet)

Einige Versionen von iconv unter Mac OS X haben einen entsprechende Herstellung der komponierten Normalform (nfc) unterstützt (utf8-mac).

In [207]:
%%bash
#UTF-8-MAC Kodierung nicht mehr im aktuellen iconv unter Mac OS X enthalten:
iconv -l | sed 5q

ANSI_X3.4-1968 ANSI_X3.4-1986 ASCII CP367 IBM367 ISO-IR-6 ISO646-US ISO_646.IRV:1991 US US-ASCII CSASCII
UTF-8
ISO-10646-UCS-2 UCS-2 CSUNICODE
UCS-2BE UNICODE-1-1 UNICODEBIG CSUNICODE11
UCS-2LE UNICODELITTLE



#### count_glyphs_mac.bash:

In [ ]:
%%bash
#!/bin/bash
iconv -f utf8-mac -t utf8 | wc -m

In [ ]:
# utf8-mac: decomposed
# utf8: composed

#### Usage:

In [189]:
%%bash
echo -n "Hallo aus München!" | iconv -f utf8 -t utf8-mac | ./count_glyphs_mac.bash

18


### Lösung 2a: count_glyphs_nfc.py mit unicodedata.normalize

Diese Lösung mit Python transformiert (genau wie die bash-Lösungen zuvor), das Input in die komponierte Normalform und berechnet anschließend die Länge des UTF-8-kodierten Strings:


#### count_glyphs_nfc.py:

In [ ]:
#!/usr/bin/env python
import sys
import unicodedata

print(len(unicodedata.normalize('NFC', sys.stdin.read())))

In [ ]:
# NFD: decomposed
# NFC: composed


Verwendung eines Hilfprogramms `decompose.py` (`'NFD` statt `'NFC'`), um die Dekomponierung zu erzwingen (analog zur Verwendung von `nfd` mit uconv oben):

#### decompose.py:

In [ ]:
#!/usr/bin/env python
import sys
import unicodedata

print(unicodedata.normalize('NFD', sys.stdin.read()), end='')

In [197]:
%%bash
echo -n "Hallo aus München!" | ./decompose.py | wc -m

      19


In [196]:
%%bash
#könnte auch als Python One-Liner geschrieben werden:
echo -n "Hallo aus München!" | 
python -c "import sys, unicodedata; print(unicodedata.normalize('NFD', sys.stdin.read()), end='')" | 
wc -m

      19


#### Usage:

In [198]:
%%bash
echo -n "Hallo aus München!" | ./decompose.py | ./count_glyphs_nfc.py

18


### Erläuterung:

#### composed (Default Encoding):

In [138]:
%%bash
echo -n "ü" | hexdump -C

00000000  c3 bc                                             |ü|
00000002


In [140]:
%%bash
#count chars
echo -n "ü" | wc -m 

       1


In [141]:
%%bash
#count bytes
echo -n "ü" | wc -c

       2


#### decomposed:

In [142]:
%%bash
# u = 75 (ASCII-Zeichen)
echo -n "ü" | ./decompose.py | hexdump -C

00000000  75 cc 88                                          |u?.|
00000003


In [143]:
%%bash
#count chars (u + combining character)
echo -n "ü" | ./decompose.py | wc -m 

       2


In [145]:
%%bash
#count bytes
echo -n "ü" | ./decompose.py | wc -c 

       3


### Lösung 2b: count_glyphs.py mit unicodedata.combining

Diese Lösung zählt nur nicht kombinierende Zeichen:

#### count_glyphs.py:

In [ ]:
#!/usr/bin/env python3
import unicodedata
import sys

count = 0
other = 0
for chr in sys.stdin.read():
    x = unicodedata.combining(chr)
    if x == 0:
        count += 1
    else: 
        other += 1
print("Buchstaben: ", count)
print("kombinierende Zeichen: ", other)

#### Usage:

In [164]:
%%bash
echo -n "ü" | ./decompose.py | ./count_glyphs.py

Buchstaben:  1
kombinierende Zeichen:  1


#### nicht alle kombinierenden Unicode-Characters werden erkannt:

In [165]:
%%bash
#Flagge Andorra = Länderkennzeichen AD (2 kombinierende Unicodecharacters, werden durch 1 Glyphe dargestellt)
echo -n "🇦🇩" | ./decompose.py | ./count_glyphs.py

Buchstaben:  2
kombinierende Zeichen:  0


In [166]:
import unicodedata
nfc = unicodedata.normalize('NFC', "🇦🇩")
print(len(nfc))

2


In [210]:
%%bash
echo -n "🇦🇩" | hexdump -C

00000000  f0 9f 87 a6 f0 9f 87 a9                           |?..??..?|                    
0000008


In [168]:
"🇦🇩".encode() #A-Ländercode D-Ländercode (combining character)

b'\xf0\x9f\x87\xa6\xf0\x9f\x87\xa9'

## 2. Aufgabe `pchars` (Encoding)
Schreiben Sie ein Programm (awk, Shell-Skript oder Python), das für
jedes Zeichen in einem Text zeilenweise, das Zeichen, seine
Unicodenummer und seine Darstellung als UTF-8 kodiertes Zeichen
(hexadezimal) ausgibt. Das Programm soll von `stdin` lesen und die
Anzahl der Zeichen nach `stdout` schreiben.

```bash
$ echo 'Aü' | ./pchars.py
A U+0041 0x41
ü U+00FC 0xc3bc
$
```

#### pchars.py:

In [212]:
#!/usr/bin/env python3
import sys

for c in sys.stdin.read():
    print(f"{c} U+{ord(c):04X}", end=" ") #Integer-Wert von c (ord()) in hexadezimal umwandeln (mit format-String)
    bs = c.encode(encoding='UTF-8') #umwandeln UTF-8-Kodierung in Byte-Array
    print("0x", end="")
    for b in bs:
        print(f"{b:02x}", end="") # Ausgabe hexadezimale UTF-8-Kodierung
    print()

#### Usage:

In [213]:
%%bash
echo -n "Mü" | ./pchars.py

M U+004D 0x4d
ü U+00FC 0xc3bc


In [214]:
%%bash
echo -n "Mü" | ./decompose.py | ./pchars.py

M U+004D 0x4d
u U+0075 0x75
̈ U+0308 0xcc88


In [215]:
%%bash
echo -n "🇦🇩" | ./pchars.py

🇦 U+1F1E6 0xf09f87a6
🇩 U+1F1E9 0xf09f87a9


In [216]:
%%bash
echo -n "🇦🇩" | ./decompose.py | ./pchars.py

🇦 U+1F1E6 0xf09f87a6
🇩 U+1F1E9 0xf09f87a9


## 3. Aufgabe `gres` (sed)
Schreiben Sie ein Skript `gres.bash`, das für einen regulären Ausdruck
RE (im ERE-Format), eine Ersetzung REPL und einem Dateipfad FILE von
der Kommandozeile, diejenigen Zeilen in der Datei FILE ausgibt, auf
die RE passt. Dabei soll jedes Vorkommen von RE in den ausgegebenen
Zeilen durch REPL ersetzt werden.
```bash
$cat file.txt
Fuchs / du
hast die
Gans gestohlen.
./gres.bash '[A-Z]' '00' file.txt
00uchs / du
00ans gestohlen.
$
```

Wegen den übergebenen regulären Ausdrücken auf der Kommandozeile
können Fehler im Skript auftreten.  Welche Fehler können auftreten und
wie kann man diese Fehler beheben?


#### gres.bash:

In [ ]:
%%bash

#!/bin/bash
# sed & awk 2nd Edition, Dale Dougherty & Arnold Robbins, O'Reilly

if [[ $# -lt 3 ]]; then
    echo "Usage $0 PATTERN REPLACEMENT FILE" >&2
    exit 1
fi

pattern=$1
replacement=$2

if [[ -f "$3" ]]; then
    file=$3
else
    echo $3 is not a file. >&2
    exit 1
fi

## LOESUNG A:
#sed -n -e "s/$pattern/$replacement/gp" "$file"

## LOESUNG B (notwendig, wenn "/" in pattern oder replacement):
A="$(echo | tr '\012' '\001')" #Ersatz NEWLINE mit NUL (nicht-druckbares Steuerzeichen als Separator als universelle Lösung)
sed -n -e "s$A$pattern$A$replacement${A}gp" "$file"

#### Usage:

In [ ]:
%%bash
./gres.bash

Usage ./gres.bash PATTERN REPLACEMENT FILE


In [ ]:
%%bash
./gres.bash e a file.txt

hast dia
Gans gastohlan


In [ ]:
%%bash
./gres.bash '[A-Z]' '00' file.txt

00uchs / du
00ans gestohlen


In [ ]:
%%bash
./gres.bash '/' '00' file.txt

Fuchs 00 du


### Erläuterung für eigenes Trennerzeichen:

In [ ]:
%%bash
#Fehler bei Versuch, "/" zu ersetzen ohne neues Trennerzeichen (sed -n -e "s/$pattern/$replacement/gp" "$file"):
./gres.bash '/' '00' file.txt

sed: 1: "s///00/gp
": bad flag in substitute command: '0'


In [ ]:
%%bash
echo -e "1/2" | sed -n -e s///_/gp

sed: 1: "s///_/gp
": bad flag in substitute command: '_'


In [ ]:
%%bash
echo -e "1/2" | sed -n -e s!/!_!gp

1_2



### Erläuterungen zum sed-Befehl:

`sed -n -e s/pattern/replacement/gp`

- `s/.../.../g` globale Ersetzung (alle Matches in einer Zeile)
- `-n`-Option zusammen mit `p`-Befehl: druckt nur die Zeilen aus, in denen Ersetzung vorgenommen wurde
  - (Default-Verhalten sed: Befehl wie Substitute auf jeder Zeile anwenden und Ergebnis ausgeben)
- `-e`-Option für Verkettung von sed-Befehlen, hier nicht unbedingt nötig

#### Beispiele:

In [ ]:
%%bash
#einfache Ersetzung
echo -e "foo\nfii" | sed s/o/e/

feo
fii


In [ ]:
%%bash
#globale Ersetzung
echo -e "foo\nfii" | sed s/o/e/g

fee
fii


In [ ]:
%%bash
#mit n-Option: Unterdrückung Ausgabe
echo -e "foo\nfii" | sed -n s/o/e/g

In [ ]:
%%bash
#mit zusätzlichem p: print nur der Zeilen mit Ersetzung
echo -e "foo\nfii" | sed -n s/o/e/gp

fee


In [ ]:
%%bash
#Beispiel für Verkettung (Beschränkung auf erste Zeile mit 1q):
echo -e "foo\nfoo" | sed -e s/o/e/g -e 1q

fee
